In [1]:
from DEA import *
from pandas import ExcelWriter

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

%load_ext nb_black

<IPython.core.display.Javascript object>

# Efficiency Measurement

In [2]:
def read_excel(dir_data, xls, sheet_name):
    """
    read input-output data from an excel

    the *dir_data* argument is the directory stored data
    the *xls* argument is the file name of the excel
    the *sheet_name* argument is the name of the excel sheet
    """
    os.chdir(dir_data)
    dea_inout = pd.read_excel(xls, sheet_name=sheet_name, header=0)
    return dea_inout

<IPython.core.display.Javascript object>

In [3]:
def read_inoutData(dir_data, xls, sheet_name, columns):
    """
    read input output data by columns

    the *dir_data* argument is the directory stored data
    the *xls* argument is the file name of the excel
    the *sheet_name* argument is the name of the excel sheet
    the *columns* argument is the names of columns to be read
    """
    data = read_excel(dir_data, xls, sheet_name)
    data_DEA = data.loc[:, columns]
    return data_DEA

<IPython.core.display.Javascript object>

In [4]:
dir_data = r"D:\tencent_files\chrome_Download\Research\DEA\DEA_RCEP\Data"
xls = "Data.xlsx"
sheet_name = "DEA_data"

<IPython.core.display.Javascript object>

In [5]:
# DEA parameters and data

input_columns = ["Population", "Capital", "Energy_consume"]
output_columns = ["GDP"]
undout_columns = ["CO2_emission"]
weight_vector = [0, 0, 1 / 3, 1 / 3, 1 / 3]
name_columns = ["Country", "Region", "year"]

<IPython.core.display.Javascript object>

In [6]:
def DEA_calc(
    dir_data=dir_data,
    xls=xls,
    sheet_name=sheet_name,
    weight_vector=weight_vector,
    undout_columns=undout_columns,
    disp="weak disposability",
    returns="CRS",
):
    data_in = read_inoutData(dir_data, xls, sheet_name, input_columns)
    data_out = read_inoutData(dir_data, xls, sheet_name, output_columns)
    data_undout = read_inoutData(dir_data, xls, sheet_name, undout_columns)
    names = read_inoutData(dir_data, xls, sheet_name, name_columns).reset_index(
        drop=True
    )

    solve = DEAProblem(
        data_in, data_out, data_undout, weight_vector, disp=disp, returns=returns
    ).solve()

    status = pd.DataFrame.from_dict(solve[0], orient="index", columns=["status"])
    efficiency = pd.DataFrame.from_dict(
        solve[1], orient="index", columns=["efficiency"]
    )
    weight = pd.DataFrame.from_dict(solve[2], orient="index")

    results = pd.concat([names, status, efficiency, weight], axis=1)
    return results.round(decimals=4)

<IPython.core.display.Javascript object>

In [7]:
os.getcwd()

'C:\\Users\\sheep\\Codes'

<IPython.core.display.Javascript object>

In [30]:
weight_vector_CEE = [0, 0, 0, 0, 1]  # carbon emission efficiency
weight_vector_CEP = [0, 0, 0, 1 / 2, 1 / 2]  # carbon economic performance
weight_vector_CEEP = [0, 0, 1 / 3, 1 / 3, 1 / 3]  # carbon energy economic performance
weight_vector_CPP = [
    1 / 9,
    1 / 9,
    1 / 9,
    1 / 3,
    1 / 3,
]  # carbon energy economic performance

<IPython.core.display.Javascript object>

In [31]:
DEA_results_CEE = DEA_calc(weight_vector=weight_vector_CEE)
DEA_results_CEP = DEA_calc(weight_vector=weight_vector_CEP)
DEA_results_CEEP = DEA_calc(weight_vector=weight_vector_CEEP)
DEA_results_CPP = DEA_calc(weight_vector=weight_vector_CPP)

<IPython.core.display.Javascript object>

In [32]:
with ExcelWriter("DEA_results.xlsx") as writer:
    DEA_results_CEE.to_excel(writer, sheet_name="CEE")
    DEA_results_CEP.to_excel(writer, sheet_name="CEP")
    DEA_results_CEEP.to_excel(writer, sheet_name="CEEP")
    DEA_results_CPP.to_excel(writer, sheet_name="CPP")

<IPython.core.display.Javascript object>

# Radar Diagram

In [11]:
def read_expan_fac(sheet_name, file_name="DEA_results.xlsx"):
    df = pd.read_excel(file_name, sheet_name=sheet_name)
    df_name = df.iloc[:, :4]
    df_expanFac = df.iloc[:, -5:]
    df_expan_fac = pd.concat([df_name, df_expanFac], axis=1)
    return df_expan_fac

<IPython.core.display.Javascript object>

In [48]:
# calc carbon emission efficiency
# df_CEE = read_expan_fac("CEE")
# df_CEE["CEE"] = 1 - df_CEE["scalingFactor_b_0"]

# calc carbon economic performance
# df_CEP = read_expan_fac("CEP")
# df_CEP["CEP"] = (1 - df_CEP["scalingFactor_b_0"]) / (1 + df_CEP["scalingFactor_y_0"])

# calc carbon economic energy performance
# df_CEEP = read_expan_fac("CEEP")
# df_CEEP["CEEP"] = (
#     1 - (df_CEEP["scalingFactor_b_0"] + df_CEEP["scalingFactor_x_2"]) / 2
# ) / (1 + df_CEEP["scalingFactor_y_0"])

# calc carbon population performance
df_CPP = read_expan_fac("CPP")
df_CPP["CEE"] = 1 - df_CPP["scalingFactor_b_0"]
df_CPP["CEP"] = (1 - df_CPP["scalingFactor_b_0"]) / (1 + df_CPP["scalingFactor_y_0"])
df_CPP["CEEP"] = (
    1 - (df_CPP["scalingFactor_b_0"] + df_CPP["scalingFactor_x_2"]) / 2
) / (1 + df_CPP["scalingFactor_y_0"])

df_CPP["CPP"] = ((1 - df_CPP["scalingFactor_b_0"]) + (1 - df_CPP["scalingFactor_x_0"]))/2
df_CPP["CPI"] = (1 - df_CPP["scalingFactor_b_0"]) / (1 - df_CPP["scalingFactor_x_0"])


<IPython.core.display.Javascript object>

In [13]:
def performance_year(df_DEA, year):
    """
    This function get AEE of each year so to draw radar graph

    screen_eff(AEE_df,year):

        Return
        ------
        eff_year: DataFrame, AEE of each DMU in the selected year

        Parameters
        ----------
        AEE_df: DataFrame, efficiency  results of the global DEA model
        year: int, the year selected for drawing radar graph

    """
    eff_year = df_DEA[df_DEA.loc[:, "year"] == year]
    return eff_year

<IPython.core.display.Javascript object>

In [15]:
def Line(line_style, line_color, line_width):
    """
    This function generate a plotly line object
    """
    Line = go.scatterpolar.Line(dash=line_style, color=line_color, width=line_width)
    return Line

<IPython.core.display.Javascript object>

In [16]:
colors = plt.get_cmap("tab20", 18).colors
colors[2] = [
    0.9,
    0.498,
    0.054,
    1.0,
]  # 因为原始的[1.0, 0.498, 0.054, 1.0]无法正确显示颜色，故改成[0.9, 0.498, 0.054, 1.0]
colors[3] = [0.9, 0.733, 0.470, 1.0]
color_line = colors
color_fill = [np.append(i[:3], 0.01) for i in colors]
colors_line = ["rgba" + str(tuple(i)) for i in color_line]
colors_fill = ["rgba" + str(tuple(i)) for i in color_fill]

<IPython.core.display.Javascript object>

In [28]:
def radar_chart(
    performance_df,  # options: df_CEE, df_CEP, df_CEEP, df_CPP
    performance_index,  # options: "CEE", "CEP", "CEEP", "CPP"
    year_range=range(2000, 2018),  # year range of the observation
    colors_line=colors_line,
    colors_fill=colors_fill,
):
    layout = go.Layout(
        polar={
            "bgcolor": "rgba(0,0,0,0)",  # set background color
            "gridshape": "linear",  # set the grid style of the radar
            "radialaxis": {"showticklabels": False, "gridcolor": "black"},  # grid color
            "angularaxis": {
                "linecolor": "black",
                "linewidth": 4,
                "gridcolor": "black",
            },
        },
        plot_bgcolor="rgba(0,0,0,0)",
    )
    fig = go.Figure(layout=layout)

    for year, color_line, color_fill in zip(year_range, colors_line, colors_fill):
        performance_year_df = performance_year(
            performance_df, year
        )  # generate data of each year for plotting
        performance_year_df = performance_year_df.append(
            performance_year_df.iloc[0]
        )  # to close the line, We need append the first element to the last
        fig.add_trace(
            go.Scatterpolar(
                r=performance_year_df.loc[
                    :, performance_index
                ],  # set the data to plot radar chart
                theta=performance_year_df.loc[
                    :, "Country"
                ],  # set the category name of each
                line=Line("dash", color_line, 3),  # set the line aesthetics
                fillcolor=color_fill,  # set the fill color
                fill="toself",  # fill the inner part of the content
                name="Year " + str(year),  # set the name of the series of data
            )
        )

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True, range=[0, max(performance_df.loc[:, performance_index])]
            )
        ),  # set the visibility of radial axis & the range of polar axis
        showlegend=True,  # set the visibility of legend
        legend=dict(y=1.1, x=0.9),  # set the position of legend
        #         width=800, used for displaying a full legend
        #         height=800,
    )
    fig.show()
    fig.write_image(
        os.path.join(
            r"D:\tencent_files\chrome_Download\Research\DEA\DEA_RCEP\Graph",
            performance_index + "_radar.png",
        )
    )  # write_image to export high-resolution chart, default export engine: orca

<IPython.core.display.Javascript object>

In [43]:
radar_chart(df_CPP, "CEE")

<IPython.core.display.Javascript object>

In [52]:
radar_chart(df_CPP, "CPP")

<IPython.core.display.Javascript object>

In [53]:
radar_chart(df_CPP, "CPI")

<IPython.core.display.Javascript object>

# Prepare Stata data

In [51]:
df = pd.concat(
    [
        df_CPP.iloc[:, :4],
        df_CPP["CEE"],
        df_CPP["CEP"],
        df_CPP["CEEP"],
        df_CPP["CPP"],
        df_CPP["CPI"],
    ],
    axis=1,
)
df.to_excel("carbon_performance.xlsx")

<IPython.core.display.Javascript object>